In [5]:
!pip install -q torch torchvision decord transformers einops timm sentencepiece
!pip install -q 'accelerate>=0.26.0'

In [2]:
# Function to create the image transformation pipeline
def build_transform(input_size):
    transform = T.Compose([
        T.Lambda(lambda img: img.convert('RGB') if img.mode != 'RGB' else img),
        T.Resize((input_size, input_size), interpolation=InterpolationMode.BICUBIC),
        T.ToTensor(),
        T.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD)
    ])
    return transform

# Function to find closest aspect ratio
def find_closest_aspect_ratio(aspect_ratio, target_ratios, width, height, image_size):
    best_ratio_diff = float('inf')
    best_ratio = (1, 1)
    area = width * height
    for ratio in target_ratios:
        target_aspect_ratio = ratio[0] / ratio[1]
        ratio_diff = abs(aspect_ratio - target_aspect_ratio)
        if ratio_diff < best_ratio_diff:
            best_ratio_diff = ratio_diff
            best_ratio = ratio
        elif ratio_diff == best_ratio_diff:
            if area > 0.5 * image_size * image_size * ratio[0] * ratio[1]:
                best_ratio = ratio
    return best_ratio

# Function for dynamic image preprocessing
def dynamic_preprocess(image, min_num=1, max_num=6, image_size=448, use_thumbnail=False):
    orig_width, orig_height = image.size
    aspect_ratio = orig_width / orig_height

    # Calculate the target aspect ratio
    target_ratios = set((i, j) for n in range(min_num, max_num + 1)
                        for i in range(1, n + 1)
                        for j in range(1, n + 1)
                        if i * j <= max_num and i * j >= min_num)
    target_ratios = sorted(target_ratios, key=lambda x: x[0] * x[1])

    target_aspect_ratio = find_closest_aspect_ratio(aspect_ratio, target_ratios, orig_width, orig_height, image_size)
    target_width, target_height = image_size * target_aspect_ratio[0], image_size * target_aspect_ratio[1]
    blocks = target_aspect_ratio[0] * target_aspect_ratio[1]

    # Resize the image
    resized_img = image.resize((target_width, target_height))
    processed_images = []
    for i in range(blocks):
        box = (
            (i % (target_width // image_size)) * image_size,
            (i // (target_width // image_size)) * image_size,
            ((i % (target_width // image_size)) + 1) * image_size,
            ((i // (target_width // image_size)) + 1) * image_size
        )
        split_img = resized_img.crop(box)
        processed_images.append(split_img)

    if use_thumbnail and len(processed_images) != 1:
        thumbnail_img = image.resize((image_size, image_size))
        processed_images.append(thumbnail_img)

    return processed_images

# Function to load and process the image
def load_image(image_file, input_size=448, max_num=6):
    image = Image.open(image_file).convert('RGB')
    transform = build_transform(input_size=input_size)
    images = dynamic_preprocess(image, image_size=input_size, use_thumbnail=True, max_num=max_num)
    pixel_values = [transform(image).to(torch.float16) for image in images]  # Use float16 precision
    pixel_values = torch.stack(pixel_values).to(device)  # Move tensor to device
    return pixel_values

In [14]:
# Function to create the image transformation pipeline
def build_transform(input_size):
    transform = T.Compose([
        T.Lambda(lambda img: img.convert('RGB') if img.mode != 'RGB' else img),
        T.Resize((input_size, input_size), interpolation=InterpolationMode.BICUBIC),
        T.ToTensor(),
        T.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD)
    ])
    return transform

# Function to find closest aspect ratio
def find_closest_aspect_ratio(aspect_ratio, target_ratios, width, height, image_size):
    best_ratio_diff = float('inf')
    best_ratio = (1, 1)
    area = width * height
    for ratio in target_ratios:
        target_aspect_ratio = ratio[0] / ratio[1]
        ratio_diff = abs(aspect_ratio - target_aspect_ratio)
        if ratio_diff < best_ratio_diff:
            best_ratio_diff = ratio_diff
            best_ratio = ratio
        elif ratio_diff == best_ratio_diff:
            if area > 0.5 * image_size * image_size * ratio[0] * ratio[1]:
                best_ratio = ratio
    return best_ratio

# Function for dynamic image preprocessing
def dynamic_preprocess(image, min_num=1, max_num=6, image_size=448, use_thumbnail=False):
    orig_width, orig_height = image.size
    aspect_ratio = orig_width / orig_height

    # Calculate the target aspect ratio
    target_ratios = set((i, j) for n in range(min_num, max_num + 1)
                        for i in range(1, n + 1)
                        for j in range(1, n + 1)
                        if i * j <= max_num and i * j >= min_num)
    target_ratios = sorted(target_ratios, key=lambda x: x[0] * x[1])

    target_aspect_ratio = find_closest_aspect_ratio(aspect_ratio, target_ratios, orig_width, orig_height, image_size)
    target_width, target_height = image_size * target_aspect_ratio[0], image_size * target_aspect_ratio[1]
    blocks = target_aspect_ratio[0] * target_aspect_ratio[1]

    # Resize the image
    resized_img = image.resize((target_width, target_height))
    processed_images = []
    for i in range(blocks):
        box = (
            (i % (target_width // image_size)) * image_size,
            (i // (target_width // image_size)) * image_size,
            ((i % (target_width // image_size)) + 1) * image_size,
            ((i // (target_width // image_size)) + 1) * image_size
        )
        split_img = resized_img.crop(box)
        processed_images.append(split_img)

    if use_thumbnail and len(processed_images) != 1:
        thumbnail_img = image.resize((image_size, image_size))
        processed_images.append(thumbnail_img)
    
    return processed_images

# Function to load and process the image
def load_image(image_file, input_size=448, max_num=6):
    image = Image.open(image_file).convert('RGB')
    transform = build_transform(input_size=input_size)
    images = dynamic_preprocess(image, image_size=input_size, use_thumbnail=True, max_num=max_num)
    pixel_values = [transform(image) for image in images]
    pixel_values = torch.stack(pixel_values)
    return pixel_values


In [4]:
# Load the model and tokenizer with float16 precision and offloading
path = 'OpenGVLab/InternVL2-8B'
model = AutoModel.from_pretrained(
    path,
    torch_dtype=torch.float16,  # Ensure FP16 precision for consistent processing
    low_cpu_mem_usage=True,
    trust_remote_code=True,
    device_map="auto",           # Offloads as needed
    offload_buffers=True         # Enables offloading of non-critical buffers
).eval()

tokenizer = AutoTokenizer.from_pretrained(path, trust_remote_code=True)

# Set generation configuration
generation_config = dict(
    num_beams=1,
    max_new_tokens=1024,
    do_sample=False,
)

# Load and preprocess an example image, moving pixel values to GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

/opt/conda/lib/python3.10/site-packages/accelerate/utils/modeling.py:1390: UserWarning: Current model requires 402656256 bytes of buffer for offloaded layers, which seems does not fit any GPU's remaining memory. If you are experiencing a OOM later, please consider using offload_buffers=True.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the cpu.


In [6]:
# Ensure pixel values use float16 precision to avoid type mismatch errors
pixel_values = load_image('/mnt/code/test_l1.png', max_num=6)
# Run the model inference
question = 'Analyze this graph'
response, history = model.chat(tokenizer, pixel_values, question, generation_config, history=None, return_history=True)

print(f'User: {question}')
print(f'Assistant: {response}')

User: Analyze this graph
Assistant: This graph shows the temperature over a period of 31 days. Here's a detailed analysis:

1. **Initial Phase (Days 1-5):**
   - The temperature starts at around 7°C and shows a gradual decline.
   - By day 5, the temperature drops to approximately 2°C.

2. **Rapid Drop (Days 5-7):**
   - There is a sharp decline in temperature, dropping from around 2°C to a low of about 1°C by day 7.

3. **Recovery Phase (Days 7-10):**
   - The temperature begins to recover, rising back up to around 6°C by day 10.

4. **Fluctuating Phase (Days 10-20):**
   - The temperature fluctuates significantly during this period. It rises to a peak of about 11°C around day 11, then drops back down to around 6°C by day 14.
   - There are several peaks and troughs, with the temperature oscillating between 6°C and 11°C.

5. **Stable Phase (Days 20-25):**
   - The temperature stabilizes around 6°C to 8°C during this period, with minor fluctuations.

6. **Final Increase (Days 25-31):**

In [7]:
# Load and preprocess another example image
pixel_values = load_image('/mnt/code/test_b2.png', max_num=6)
# Run the model inference
question = 'Analyze this graph'
response, history = model.chat(tokenizer, pixel_values, question, generation_config, history=None, return_history=True)

print(f'User: {question}')
print(f'Assistant: {response}')

User: Analyze this graph
Assistant: This bar graph is titled Tslesie cna Aeihndnaes. The y-axis measures % hei 50 setter dniae NEASYS while the x-axis measures Site, Ischaemia, Neuropathy, Bacterial infection, Area, and Depth. The highest value is 84.6% for Neuropathy. The lowest value is 11.8% for Depth.


In [8]:
# Load and preprocess another example image
pixel_values = load_image('/mnt/code/test_p3.png', max_num=6)
# Run the model inference
question = 'Analyze this graph'
response, history = model.chat(tokenizer, pixel_values, question, generation_config, history=None, return_history=True)

print(f'User: {question}')
print(f'Assistant: {response}')

User: Analyze this graph
Assistant: The pie chart titled "Quina és la Taxa d'ocupació?" (What is the occupancy rate?) shows the distribution of occupancy rates among three categories: "Ocupat" (occupied), "Aturat" (vacant), and "Inactiu" (inactive). Here's a detailed analysis:

1. **Ocupat (Occupied)**:
   - The largest segment of the pie chart, representing 76.6% of the total.
   - This indicates that the majority of the units or spaces are occupied.

2. **Aturat (Vacant)**:
   - The second-largest segment, representing 15.9% of the total.
   - This suggests that a significant portion of the units or spaces are vacant.

3. **Inactiu (Inactive)**:
   - The smallest segment, representing 6.5% of the total.
   - This indicates that a smaller portion of the units or spaces are inactive.

### Key Insights:
- The occupancy rate is predominantly high, with 76.6% of the units or spaces being occupied.
- A notable portion, 15.9%, is vacant, which could be due to various reasons such as mainten